In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver

class EasyApplyLinkedin:
    
    
    def __init__(self):
        driver = webdriver.chrome()
        self.driver() 
       # """Parameter initialization"""

        #self.email = data['email']
        #self.password = data['password']
        #self.keywords = data['keywords']
        #self.location = data['location']
       
    def login_linkedin(self):
        """This function logs into your personal LinkedIn profile"""
        #self.driver = webdriver.Chrome()

        # go to the LinkedIn login url
        self.driver.get("https:www.google.com")

        # introduce email and password and hit enter
    
        
        login_email = self.driver.find_element(By.NAME,"session_key")
        #login_email.clear()
        login_email.send_keys('mnshreyas615@gmail.com')
        login_pass = self.driver.find_element(By.NAME,"session_password")
        #login_pass = self.driver.find_element_by_name('')
        login_pass.clear()
        login_pass.send_keys('Shreyash@2625')
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """This function goes to the 'Jobs' section a looks for all the jobs that matches the keywords and location"""

        # go to Jobs
        jobs_link = self.driver.find_element(By.LINK_TEXT, "Jobs")
        #jobs_link = self.driver.find_element_by_link_text('Jobs')
        jobs_link.click()

        # search based on keywords and location and hit enter
        search_keywords = self.driver.find_element(By.CSS_SELECTOR, "Title, skill, or company")
        #search_keywords = self.driver.find_element_by_css_selector(".jobs-search-box__text-input[aria-label='Search jobs']")
        #search_keywords.clear()
        search_keywords.send_keys(self.keywords)
        search_location = self.driver.find_element(By.ID, "jobs-search-box-location-id-ember268")
        #search_location = self.driver.find_element_by_css_selector(".jobs-search-box__text-input[aria-label='Search location']")
        search_location.clear()
        search_location.send_keys(self.location)
        search_location.send_keys(Keys.RETURN)

    def filter(self):
        """This function filters all the job results by 'Easy Apply'"""

        # select all filters, click on Easy Apply and apply the filter
        all_filters_button = self.driver.find_element_by_xpath("//button[@data-control-name='all_filters']")
        all_filters_button.click()
        time.sleep(1)
        easy_apply_button = self.driver.find_element_by_xpath("//label[@for='f_LF-f_AL']")
        easy_apply_button.click()
        time.sleep(1)
        apply_filter_button = self.driver.find_element_by_xpath("//button[@data-control-name='all_filters_apply']")
        apply_filter_button.click()

    def find_offers(self):
        """This function finds all the offers through all the pages result of the search and filter"""

        # find the total amount of results (if the results are above 24-more than one page-, we will scroll trhough all available pages)
        total_results = self.driver.find_element_by_class_name("display-flex.t-12.t-black--light.t-normal")
        total_results_int = int(total_results.text.split(' ',1)[0].replace(",",""))
        print(total_results_int)

        time.sleep(2)
        # get results for the first page
        current_page = self.driver.current_url
        results = self.driver.find_elements_by_class_name("occludable-update.artdeco-list_item--offset-4.artdeco-list_item.p0.ember-view")

        # for each job add, submits application if no questions asked
        for result in results:
            hover = ActionChains(self.driver).move_to_element(result)
            hover.perform()
            titles = result.find_elements_by_class_name('job-card-search_title.artdeco-entity-lockup_title.ember-view')
            for title in titles:
                self.submit_apply(title)

        # if there is more than one page, find the pages and apply to the results of each page
        if total_results_int > 24:
            time.sleep(2)

            # find the last page and construct url of each page based on the total amount of pages
            find_pages = self.driver.find_elements_by_class_name("artdeco-pagination_indicator.artdeco-pagination_indicator--number")
            total_pages = find_pages[len(find_pages)-1].text
            total_pages_int = int(re.sub(r"[^\d.]", "", total_pages))
            get_last_page = self.driver.find_element_by_xpath("//button[@aria-label='Page "+str(total_pages_int)+"']")
            get_last_page.send_keys(Keys.RETURN)
            time.sleep(2)
            last_page = self.driver.current_url
            total_jobs = int(last_page.split('start=',1)[1])

            # go through all available pages and job offers and apply
            for page_number in range(25,total_jobs+25,25):
                self.driver.get(current_page+'&start='+str(page_number))
                time.sleep(2)
                results_ext = self.driver.find_elements_by_class_name("occludable-update.artdeco-list_item--offset-4.artdeco-list_item.p0.ember-view")
                for result_ext in results_ext:
                    hover_ext = ActionChains(self.driver).move_to_element(result_ext)
                    hover_ext.perform()
                    titles_ext = result_ext.find_elements_by_class_name('job-card-search_title.artdeco-entity-lockup_title.ember-view')
                    for title_ext in titles_ext:
                        self.submit_apply(title_ext)
        else:
            self.close_session()

    def submit_apply(self,job_add):
        """This function submits the application for the job add found"""

        print('You are applying to the position of: ', job_add.text)
        job_add.click()
        time.sleep(2)
        
        # click on the easy apply button, skip if already applied to the position
        try:
            in_apply = self.driver.find_element_by_xpath("//button[@data-control-name='jobdetails_topcard_inapply']")
            in_apply.click()
        except NoSuchElementException:
            print('You already applied to this job, go to next...')
            pass
        time.sleep(1)

        # try to submit if submit application is available...
        try:
            submit = self.driver.find_element_by_xpath("//button[@data-control-name='submit_unify']")
            submit.send_keys(Keys.RETURN)
        
        # ... if not available, discard application and go to next
        except NoSuchElementException:
            print('Not direct application, going to next...')
            try:
                discard = self.driver.find_element_by_xpath("//button[@data-test-modal-close-btn]")
                discard.send_keys(Keys.RETURN)
                time.sleep(1)
                discard_confirm = self.driver.find_element_by_xpath("//button[@data-test-dialog-primary-btn]")
                discard_confirm.send_keys(Keys.RETURN)
                time.sleep(1)
            except NoSuchElementException:
                pass

        time.sleep(1)

    def close_session(self):
        """This function closes the actual session"""
        
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers"""

        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)
        self.job_search()
        time.sleep(5)
        self.filter()
        time.sleep(2)
        self.find_offers()
        time.sleep(2)
        self.close_session()


#if _name_ == '_main_':

#with open('config.json') as config_file:
    #data = json.load(config_file)

bot = EasyApplyLinkedin()
bot.apply()

TypeError: 'module' object is not callable

In [15]:
##This code is working for my job icon selection
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """This function logs into your personal LinkedIn profile"""
        self.driver.get("https://www.linkedin.com/login")

        login_email = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_key"))
        )
        login_email.send_keys(self.email)

        login_pass = self.driver.find_element(By.NAME, "session_password")
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """This function goes to the 'Jobs' section a looks for all the jobs that matches the keywords and location"""
        jobs_link = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))
        )
        jobs_link.click()

        search_keywords = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.PLACEHOLDER,"Search by title, skill, or company" ))
        )
        search_keywords.send_keys(self.keywords)

        search_location = self.driver.find_element(By.CSS_SELECTOR, "input[aria-label='Search location']")
        search_location.clear()
        search_location.send_keys(self.location)
        search_location.send_keys(Keys.RETURN)

    def close_session(self):
        """This function closes the actual session"""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers"""
        self.login_linkedin()
        time.sleep(5)
        self.job_search()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


AttributeError: type object 'By' has no attribute 'PLACEHOLDER'

In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """This function logs into your personal LinkedIn profile"""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_key"))).send_keys(self.email)
        login_pass = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_password")))
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """This function goes to the 'Jobs' section and looks for all the jobs that matches the keywords and location"""
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))).click()

        # Wait and enter keywords in the search box using placeholder
        search_keywords = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Search by title, skill, or company']"))
        )
        search_keywords.clear()
        search_keywords.send_keys(self.keywords)

        # Wait and enter location
        search_location = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='City, state, or zip code']"))
        )
        search_location.clear()
        search_location.send_keys(self.location)
        search_location.send_keys(Keys.RETURN)

    def close_session(self):
        """This function closes the actual session"""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers"""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        # More actions can be added here if needed
        time.sleep(2)  # Again, consider adjusting these waits
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


JavascriptException: Message: javascript error: {"status":12,"value":"Element is not currently interactable and may not be manipulated"}
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF699E91F52+60322]
	(No symbol) [0x00007FF699E0CEC9]
	(No symbol) [0x00007FF699CC7EBA]
	(No symbol) [0x00007FF699CCDCEE]
	(No symbol) [0x00007FF699CD0641]
	(No symbol) [0x00007FF699CD06E0]
	(No symbol) [0x00007FF699D0DAEA]
	(No symbol) [0x00007FF699D3C21A]
	(No symbol) [0x00007FF699D0ADB6]
	(No symbol) [0x00007FF699D3C430]
	(No symbol) [0x00007FF699D5BC80]
	(No symbol) [0x00007FF699D3BFC3]
	(No symbol) [0x00007FF699D09617]
	(No symbol) [0x00007FF699D0A211]
	GetHandleVerifier [0x00007FF69A1A94AD+3301629]
	GetHandleVerifier [0x00007FF69A1F36D3+3605283]
	GetHandleVerifier [0x00007FF69A1E9450+3563680]
	GetHandleVerifier [0x00007FF699F44326+790390]
	(No symbol) [0x00007FF699E1750F]
	(No symbol) [0x00007FF699E13404]
	(No symbol) [0x00007FF699E13592]
	(No symbol) [0x00007FF699E02F

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """This function logs into your personal LinkedIn profile."""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_key"))).send_keys(self.email)
        login_pass = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_password")))
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """This function goes to the 'Jobs' section and looks for all the jobs that match the keywords and location."""
        try:
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))).click()

            # Wait for the keyword search box to be interactable
            search_keywords = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CLASS, "input[placeholder='Search by title, skill, or company']"))
            )
            search_keywords.clear()
            search_keywords.send_keys(self.keywords)

            # Wait for the location search box to be interactable
            search_location = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='City, state, or zip code']"))
            )
            search_location.clear()
            search_location.send_keys(self.location)
            search_location.send_keys(Keys.RETURN)

        except Exception as e:
            print(f"An error occurred while trying to search for jobs: {str(e)}")
            if "not interactable" in str(e):
                # Try using JavaScript to set the value if normal methods fail
                self.driver.execute_script("arguments[0].value = arguments[1];", search_keywords, self.keywords)
                self.driver.execute_script("arguments[0].value = arguments[1];", search_location, self.location)
                search_location.send_keys(Keys.RETURN)

    def close_session(self):
        """This function closes the actual session."""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers."""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        time.sleep(2)  # Again, consider adjusting these waits
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


An error occurred while trying to search for jobs: type object 'By' has no attribute 'CLASS'
End of the session, see you later!


In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """This function logs into your personal LinkedIn profile."""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_key"))).send_keys(self.email)
        login_pass = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_password")))
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """This function goes to the 'Jobs' section and looks for all the jobs that match the keywords and location."""
        try:
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))).click()
            search_box_container = WebDriverWait(self.driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "jobs-search-box__inner"))
            )

            search_keywords = search_box_container.find_element(By.CSS_SELECTOR, "input[placeholder='Search by title, skill, or company']")
            search_keywords.clear()
            search_keywords.send_keys(self.keywords)

            search_location = search_box_container.find_element(By.CSS_SELECTOR, "input[placeholder='City, state, or zip code']")
            search_location.clear()
            search_location.send_keys(self.location)
            search_location.send_keys(Keys.RETURN)

        except Exception as e:
            print(f"An error occurred while trying to search for jobs: {str(e)}")

    def close_session(self):
        """This function closes the actual session."""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers."""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        time.sleep(2)  # Again, consider adjusting these waits
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


An error occurred while trying to search for jobs: Message: javascript error: {"status":12,"value":"Element is not currently interactable and may not be manipulated"}
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF699E91F52+60322]
	(No symbol) [0x00007FF699E0CEC9]
	(No symbol) [0x00007FF699CC7EBA]
	(No symbol) [0x00007FF699CCDCEE]
	(No symbol) [0x00007FF699CD0641]
	(No symbol) [0x00007FF699CD06E0]
	(No symbol) [0x00007FF699D0DAEA]
	(No symbol) [0x00007FF699D3C21A]
	(No symbol) [0x00007FF699D0ADB6]
	(No symbol) [0x00007FF699D3C430]
	(No symbol) [0x00007FF699D5BC80]
	(No symbol) [0x00007FF699D3BFC3]
	(No symbol) [0x00007FF699D09617]
	(No symbol) [0x00007FF699D0A211]
	GetHandleVerifier [0x00007FF69A1A94AD+3301629]
	GetHandleVerifier [0x00007FF69A1F36D3+3605283]
	GetHandleVerifier [0x00007FF69A1E9450+3563680]
	GetHandleVerifier [0x00007FF699F44326+790390]
	(No symbol) [0x00007FF699E1750F]
	(No symbol) [0x00007FF699E13404]
	(No symbol) [0x00007FF699E13592]


In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """This function logs into your personal LinkedIn profile."""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_key"))).send_keys(self.email)
        login_pass = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "session_password")))
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """This function goes to the 'Jobs' section and looks for all the jobs that match the keywords and location."""
        try:
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))).click()
            search_box_container = WebDriverWait(self.driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "jobs-search-box__inner"))
            )

            search_keywords = search_box_container.find_element(By.CSS_SELECTOR, "input[placeholder='Search by title, skill, or company']")
            search_keywords.clear()
            search_keywords.send_keys(self.keywords)

            search_location = search_box_container.find_element(By.CSS_SELECTOR, "input[placeholder='City, state, or zip code']")
            search_location.clear()
            search_location.send_keys(self.location)
            search_location.send_keys(Keys.RETURN)

        except Exception as e:
            print(f"An error occurred while trying to search for jobs: {str(e)}")

    def close_session(self):
        """This function closes the actual session."""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers."""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        time.sleep(2)  # Again, consider adjusting these waits
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


An error occurred while trying to search for jobs: Message: javascript error: {"status":12,"value":"Element is not currently interactable and may not be manipulated"}
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF699E91F52+60322]
	(No symbol) [0x00007FF699E0CEC9]
	(No symbol) [0x00007FF699CC7EBA]
	(No symbol) [0x00007FF699CCDCEE]
	(No symbol) [0x00007FF699CD0641]
	(No symbol) [0x00007FF699CD06E0]
	(No symbol) [0x00007FF699D0DAEA]
	(No symbol) [0x00007FF699D3C21A]
	(No symbol) [0x00007FF699D0ADB6]
	(No symbol) [0x00007FF699D3C430]
	(No symbol) [0x00007FF699D5BC80]
	(No symbol) [0x00007FF699D3BFC3]
	(No symbol) [0x00007FF699D09617]
	(No symbol) [0x00007FF699D0A211]
	GetHandleVerifier [0x00007FF69A1A94AD+3301629]
	GetHandleVerifier [0x00007FF69A1F36D3+3605283]
	GetHandleVerifier [0x00007FF69A1E9450+3563680]
	GetHandleVerifier [0x00007FF699F44326+790390]
	(No symbol) [0x00007FF699E1750F]
	(No symbol) [0x00007FF699E13404]
	(No symbol) [0x00007FF699E13592]


In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        """Initialize the webdriver and store credentials and search parameters."""
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """Log into LinkedIn using provided credentials."""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_key"))
        ).send_keys(self.email)

        login_pass = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_password"))
        )
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """Navigate to the 'Jobs' section and perform a job search using the provided keywords and location."""
        try:
            WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))
            ).click()

            search_keywords = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Search by title, skill, or company']"))
            )
            self.driver.execute_script("arguments[0].scrollIntoView(true);", search_keywords)
            search_keywords.clear()
            search_keywords.send_keys(self.keywords)

            search_location = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='City, state, or zip code']"))
            )
            search_location.clear()
            search_location.send_keys(self.location)
            search_location.send_keys(Keys.RETURN)

        except Exception as e:
            print(f"An error occurred while trying to search for jobs: {str(e)}")

    def close_session(self):
        """Close the browser session."""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers by initiating the login, search, and session closure sequences."""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        time.sleep(2)  # Again, consider adjusting these waits
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


An error occurred while trying to search for jobs: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF699E91F52+60322]
	(No symbol) [0x00007FF699E0CEC9]
	(No symbol) [0x00007FF699CC7EBA]
	(No symbol) [0x00007FF699D17676]
	(No symbol) [0x00007FF699D1773C]
	(No symbol) [0x00007FF699D5E967]
	(No symbol) [0x00007FF699D3C25F]
	(No symbol) [0x00007FF699D5BC80]
	(No symbol) [0x00007FF699D3BFC3]
	(No symbol) [0x00007FF699D09617]
	(No symbol) [0x00007FF699D0A211]
	GetHandleVerifier [0x00007FF69A1A94AD+3301629]
	GetHandleVerifier [0x00007FF69A1F36D3+3605283]
	GetHandleVerifier [0x00007FF69A1E9450+3563680]
	GetHandleVerifier [0x00007FF699F44326+790390]
	(No symbol) [0x00007FF699E1750F]
	(No symbol) [0x00007FF699E13404]
	(No symbol) [0x00007FF699E13592]
	(No symbol) [0x00007FF699E02F9F]
	BaseThreadInitThunk [0x00007FFAC7F4257D+29]
	RtlUserThreadStart [0x00007FFAC95AAA48+40]

End of the session, see you later!


In [22]:
npm install puppeteer nodemon

SyntaxError: invalid syntax (1414564355.py, line 1)

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        """Initialize the webdriver and store credentials and search parameters."""
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """Log into LinkedIn using provided credentials."""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_key"))
        ).send_keys(self.email)

        login_pass = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_password"))
        )
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """Navigate to the 'Jobs' section and perform a job search using the provided keywords and location."""
        try:
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))).click()

            # Find the search input using the title attribute for keywords
            search_keywords = WebDriverWait(self.driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[title="Search by title, skill, or company"]'))
            )
            search_keywords.clear()
            search_keywords.send_keys(self.keywords)

            # Assume there's another input for location, using a hypothetical placeholder
            search_location = WebDriverWait(self.driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[placeholder="City, state, or zip code"]'))
            )
            search_location.clear()
            search_location.send_keys(self.location)
            search_location.send_keys(Keys.RETURN)

        except Exception as e:
            print(f"An error occurred while trying to search for jobs: {str(e)}")

    def close_session(self):
        """Close the browser session."""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers by initiating the login, search, and session closure sequences."""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        time.sleep(2)  # Again, consider adjusting these waits
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


An error occurred while trying to search for jobs: Message: javascript error: {"status":12,"value":"Element is not currently interactable and may not be manipulated"}
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF699E91F52+60322]
	(No symbol) [0x00007FF699E0CEC9]
	(No symbol) [0x00007FF699CC7EBA]
	(No symbol) [0x00007FF699CCDCEE]
	(No symbol) [0x00007FF699CD0641]
	(No symbol) [0x00007FF699CD06E0]
	(No symbol) [0x00007FF699D0DAEA]
	(No symbol) [0x00007FF699D3C21A]
	(No symbol) [0x00007FF699D0ADB6]
	(No symbol) [0x00007FF699D3C430]
	(No symbol) [0x00007FF699D5BC80]
	(No symbol) [0x00007FF699D3BFC3]
	(No symbol) [0x00007FF699D09617]
	(No symbol) [0x00007FF699D0A211]
	GetHandleVerifier [0x00007FF69A1A94AD+3301629]
	GetHandleVerifier [0x00007FF69A1F36D3+3605283]
	GetHandleVerifier [0x00007FF69A1E9450+3563680]
	GetHandleVerifier [0x00007FF699F44326+790390]
	(No symbol) [0x00007FF699E1750F]
	(No symbol) [0x00007FF699E13404]
	(No symbol) [0x00007FF699E13592]


In [25]:
import asyncio
from pyppeteer import launch
import json

async def login(page, email, password):
    await page.type('[name="session_key"]', email)
    await page.type('[name="session_password"]', password)
    await page.keyboard.press('Enter')

async def initializer(browserPath, resolution):
    browser = await launch(headless=False, executablePath=browserPath, args=[resolution], defaultViewport=None)
    page = await browser.newPage()
    pages = await browser.pages()
    if len(pages) > 1:
        await pages[0].close()
    return page, browser

async def find_target_and_type(page, target, value):
    await page.type(target, value)

async def button_click(page, selector):
    await page.waitForSelector(selector)
    button = await page.querySelector(selector)
    await button.click()

async def job_criteria_by_time(page, period_of_time):
    await button_click(page, ".search-reusables__filter-binary-toggle")
    await asyncio.sleep(2)
    await button_click(page, "ul.search-reusables__filter-list>li:nth-child(4)>div>span>button")
    if period_of_time == "Past 24 hours":
        await asyncio.sleep(2)
        await button_click(page, "form > fieldset > div.pl4.pr6 > ul > li:nth-child(4) > label")
    else:
        await asyncio.sleep(2)
        await button_click(page, "form > fieldset > div.pl4.pr6 > ul > li:nth-child(3) > label")
    await asyncio.sleep(2)
    await button_click(page, ".render-mode-BIGPIPE")

async def job_criteria_by_type(page):
    await button_click(page, 'div[id="hoverable-outlet-on-site/remote-filter-value"]+span>button')
    await asyncio.sleep(2)
    await button_click(page, "label[for^='workplaceType']")
    await asyncio.sleep(2)
    await button_click(page, ".render-mode-BIGPIPE")
    await asyncio.sleep(2)

async def fill_and_apply(page, number_of_pagination, nbr_of_offers_per_page):
    for i in range(1, number_of_pagination + 1):
        print(f"Scrolling the page N°{i}")
        for index in range(1, nbr_of_offers_per_page + 1):
            await asyncio.sleep(3)
            # Scroll to the item
            await page.evaluate('() => { document.querySelector("div[class=\"scaffold-layout__list-detail-inner\"]>section>div>ul").scrollIntoView(); }')
            print(f"Apply N°[{index}]")
            # Additional implementation needed here for clicking and handling forms

async def main():
    with open('config.json') as config_file:
        data = json.load(config_file)

    BaseURL = data['baseURL']
    email = data['email']
    password = data['password']
    browserPath = data['ChromePath']
    resolution = data['resolution']
    number_of_pagination = data['numberOfPagination']
    nbr_of_offers_per_page = data['numberOfOffersPerPage']
    period_of_time = data['Period']

    page, browser = await initializer(browserPath, resolution)
    await page.goto(BaseURL)
    await login(page, email, password)

    await job_criteria_by_time(page, period_of_time)
    await job_criteria_by_type(page)
    await fill_and_apply(page, number_of_pagination, nbr_of_offers_per_page)

    await browser.close()

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [27]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        """Initialize the webdriver and store credentials and search parameters."""
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """Log into LinkedIn using provided credentials."""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_key"))
        ).send_keys(self.email)

        login_pass = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_password"))
        )
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """Navigate to the 'Jobs' section and perform a job search using the provided keywords and location."""
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))).click()

        search_keywords = WebDriverWait(self.driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[title="Search by title, skill, or company"]'))
        )
        search_keywords.clear()
        search_keywords.send_keys(self.keywords)

        search_location = WebDriverWait(self.driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[placeholder="City, state, or zip code"]'))
        )
        search_location.clear()
        search_location.send_keys(self.location)
        search_location.send_keys(Keys.RETURN)
        self.apply_filters()

    def apply_filters(self):
        """Apply specific filters like date posted and job type if necessary."""
        # Assuming you have logic here to select the 'Easy Apply' filter
        pass

    def navigate_and_apply(self):
        """Navigate through job listings and apply."""
        # Your logic here to go through the job listings
        # Example: Click on job listing, handle application form, submit application
        pass

    def handle_pagination(self):
        """Handle pagination to go through multiple pages of job listings."""
        # Example logic to click on next page
        pass

    def close_session(self):
        """Close the browser session."""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers by initiating the login, search, and session closure sequences."""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        time.sleep(2)  # Adjusting waits dynamically based on page load
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


JavascriptException: Message: javascript error: {"status":12,"value":"Element is not currently interactable and may not be manipulated"}
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF699E91F52+60322]
	(No symbol) [0x00007FF699E0CEC9]
	(No symbol) [0x00007FF699CC7EBA]
	(No symbol) [0x00007FF699CCDCEE]
	(No symbol) [0x00007FF699CD0641]
	(No symbol) [0x00007FF699CD06E0]
	(No symbol) [0x00007FF699D0DAEA]
	(No symbol) [0x00007FF699D3C21A]
	(No symbol) [0x00007FF699D0ADB6]
	(No symbol) [0x00007FF699D3C430]
	(No symbol) [0x00007FF699D5BC80]
	(No symbol) [0x00007FF699D3BFC3]
	(No symbol) [0x00007FF699D09617]
	(No symbol) [0x00007FF699D0A211]
	GetHandleVerifier [0x00007FF69A1A94AD+3301629]
	GetHandleVerifier [0x00007FF69A1F36D3+3605283]
	GetHandleVerifier [0x00007FF69A1E9450+3563680]
	GetHandleVerifier [0x00007FF699F44326+790390]
	(No symbol) [0x00007FF699E1750F]
	(No symbol) [0x00007FF699E13404]
	(No symbol) [0x00007FF699E13592]
	(No symbol) [0x00007FF699E02F

In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import json

class EasyApplyLinkedin:
    
    def __init__(self, email, password, keywords, location):
        """Initialize the webdriver and store credentials and search parameters."""
        self.driver = webdriver.Chrome()
        self.email = email
        self.password = password
        self.keywords = keywords
        self.location = location

    def login_linkedin(self):
        """Log into LinkedIn using provided credentials."""
        self.driver.get("https://www.linkedin.com/login")
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_key"))
        ).send_keys(self.email)

        login_pass = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "session_password"))
        )
        login_pass.clear()
        login_pass.send_keys(self.password)
        login_pass.send_keys(Keys.RETURN)
    
    def job_search(self):
        """Navigate to the 'Jobs' section and perform a job search using the provided keywords and location."""
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Jobs"))).click()

        # Ensure the search field is visible and interactable
        search_keywords = WebDriverWait(self.driver, 20).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[title="Search by title, skill, or company"]'))
        )
        self.driver.execute_script("arguments[0].scrollIntoView(true);", search_keywords)
        search_keywords.clear()
        search_keywords.send_keys(self.keywords)

        # Ensure the location field is also visible and interactable
        search_location = WebDriverWait(self.driver, 20).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[placeholder="City, state, or zip code"]'))
        )
        search_location.clear()
        search_location.send_keys(self.location)
        search_location.send_keys(Keys.RETURN)

    def close_session(self):
        """Close the browser session."""
        print('End of the session, see you later!')
        self.driver.close()

    def apply(self):
        """Apply to job offers by initiating the login, search, and session closure sequences."""
        self.driver.maximize_window()
        self.login_linkedin()
        time.sleep(5)  # Consider replacing fixed sleeps with more robust wait conditions
        self.job_search()
        time.sleep(2)  # Again, consider adjusting these waits
        self.close_session()

# Usage example with external configuration loading
if __name__ == "__main__":
    with open('config.json') as config_file:
        data = json.load(config_file)
    bot = EasyApplyLinkedin(data['email'], data['password'], data['keywords'], data['location'])
    bot.apply()


WebDriverException: Message: Service C:\Users\Shreyas MN\.cache\selenium\chromedriver\win64\125.0.6422.141\chromedriver.exe unexpectedly exited. Status code was: 3221225794
